<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n30_x2_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n30_x2_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    10.867320  30.522385  ...  180.608993  -172.659234
     1.0    38.828558  38.705901  ...  443.268121   825.998407
     2.0    19.065912  42.388963  ...  263.115598  -228.202952
     3.0    22.570891  39.598609  ...  293.777262  -132.158763
     4.0    25.972302  35.787483  ...  322.540647   296.113985
...               ...        ...  ...         ...          ...
99.0 25.0   48.830285  22.017678  ...  549.343270 -5663.297019
     26.0   43.619352  13.779638  ...  493.978627  2063.374637
     27.0   17.269765  48.865489  ...  237.845971   293.180841
     28.0   13.531417  44.327993  ...  211.381289   152.643921
     29.0   22.236480  17.892346  ...  292.162709   342.066151

[3000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    10.867320  30.522385  ...  180.608993  -172.659234
     1.0    38.828558  38.705901  ...  443.268121   825.998407
     2.0    19.065912  42.388963  ...  263.115598  -228.202952
     3.0    22.570891  39.598609  ...  293.777262  -132.158763
     4.0    25.972302  35.787483  ...  322.540647   296.113985
...               ...        ...  ...         ...          ...
99.0 25.0   48.830285  22.017678  ...  549.343270 -5663.297019
     26.0   43.619352  13.779638  ...  493.978627  2063.374637
     27.0   17.269765  48.865489  ...  237.845971   293.180841
     28.0   13.531417  44.327993  ...  211.381289   152.643921
     29.0   22.236480  17.892346  ...  292.162709   342.066151

[3000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,10.867320,30.522385,14.381240,-150.627071,7.949759,180.608993,-172.659234
1,0.0,1.0,38.828558,38.705901,12.851219,824.423832,1269.266528,443.268121,825.998407
2,0.0,2.0,19.065912,42.388963,41.487465,-244.622905,34.912646,263.115598,-228.202952
3,0.0,3.0,22.570891,39.598609,29.993061,-138.682084,161.618499,293.777262,-132.158763
4,0.0,4.0,25.972302,35.787483,14.233377,303.910753,618.654632,322.540647,296.113985
...,...,...,...,...,...,...,...,...,...
2995,99.0,25.0,48.830285,22.017678,48.188771,-5677.463052,-5113.953749,549.343270,-5663.297019
2996,99.0,26.0,43.619352,13.779638,18.962015,2083.418087,2557.353265,493.978627,2063.374637
2997,99.0,27.0,17.269765,48.865489,12.673466,291.790208,531.026813,237.845971,293.180841
2998,99.0,28.0,13.531417,44.327993,44.990025,134.393026,364.025210,211.381289,152.643921


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    10.867320  30.522385  ...  180.608993  -172.659234
     1.0    38.828558  38.705901  ...  443.268121   825.998407
     2.0    19.065912  42.388963  ...  263.115598  -228.202952
     3.0    22.570891  39.598609  ...  293.777262  -132.158763
     4.0    25.972302  35.787483  ...  322.540647   296.113985
...               ...        ...  ...         ...          ...
99.0 25.0   48.830285  22.017678  ...  549.343270 -5663.297019
     26.0   43.619352  13.779638  ...  493.978627  2063.374637
     27.0   17.269765  48.865489  ...  237.845971   293.180841
     28.0   13.531417  44.327993  ...  211.381289   152.643921
     29.0   22.236480  17.892346  ...  292.162709   342.066151

[3000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
1.0,8.0,16.0,5878219.069447905,20136678.215386957,839745.5813497007,1342445.2143591305,1.71282134754449,0.44591213951226827,0.7770439302438659,Reject005=1 : Homoscedasticity
2.0,9.0,15.0,310540.99636372767,33625728.629578665,38817.62454546596,2401837.759255619,64.9686753568482,5.206471969110282e-07,0.9999997396764015,Reject005=0 : Heteroscedasticity
3.0,12.0,12.0,3804010.454198524,35147708.71733991,345819.1321998658,3195246.247030901,9.239645668835383,0.00051867914648418,0.9997406604267579,Reject005=0 : Heteroscedasticity
4.0,8.0,16.0,7005032.847651508,44034267.74353999,1000718.9782359296,2935617.849569333,3.1430450578331564,0.10519669557854483,0.9474016522107276,Reject005=1 : Homoscedasticity
5.0,8.0,16.0,284601.21286377514,9677132.088946108,40657.31612339645,645142.1392630738,17.00121371861139,0.0003741728404673683,0.9998129135797663,Reject005=0 : Heteroscedasticity
6.0,10.0,14.0,10848767.643425366,30872385.647398435,1205418.6270472629,2374798.8959537256,2.032645988802233,0.2627557431613803,0.8686221284193099,Reject005=1 : Homoscedasticity
7.0,7.0,17.0,1878947.5753184797,14862562.296476308,313157.9292197466,928910.1435297693,3.257077883947562,0.11815870115104432,0.9409206494244778,Reject005=1 : Homoscedasticity
11.0,13.0,11.0,2860280.5232827887,14477759.231502125,238356.71027356572,1447775.9231502125,5.981958396212737,0.0033464605007036408,0.9983267697496482,Reject005=0 : Heteroscedasticity
12.0,16.0,8.0,13140486.764120402,16390136.637418656,876032.4509413601,2341448.091059808,2.494601140982281,0.11389155746959534,0.9430542212652023,Reject005=1 : Homoscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,16.0,8.0,21542016.069511123,3340770.245077745,1436134.4046340748,477252.89215396356,0.31016319311041723,0.09809046237645044,Reject005=1 : Homoscedasticity
1.0,8.0,16.0,5878219.069447905,20136678.215386957,839745.5813497007,1342445.2143591305,1.71282134754449,0.44591213951226827,Reject005=1 : Homoscedasticity
2.0,9.0,15.0,310540.99636372767,33625728.629578665,38817.62454546596,2401837.759255619,64.9686753568482,5.206471969110282e-07,Reject005=0 : Heteroscedasticity
3.0,12.0,12.0,3804010.454198524,35147708.71733991,345819.1321998658,3195246.247030901,9.239645668835383,0.00051867914648418,Reject005=0 : Heteroscedasticity
4.0,8.0,16.0,7005032.847651508,44034267.74353999,1000718.9782359296,2935617.849569333,3.1430450578331564,0.10519669557854483,Reject005=1 : Homoscedasticity
5.0,8.0,16.0,284601.21286377514,9677132.088946108,40657.31612339645,645142.1392630738,17.00121371861139,0.0003741728404673683,Reject005=0 : Heteroscedasticity
6.0,10.0,14.0,10848767.643425366,30872385.647398435,1205418.6270472629,2374798.8959537256,2.032645988802233,0.2627557431613803,Reject005=1 : Homoscedasticity
7.0,7.0,17.0,1878947.5753184797,14862562.296476308,313157.9292197466,928910.1435297693,3.257077883947562,0.11815870115104432,Reject005=1 : Homoscedasticity
8.0,9.0,15.0,8767078.830776116,12725846.377352407,1095884.8538470145,908989.0269537434,0.8709295278157664,0.7807564829172796,Reject005=1 : Homoscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    51
Reject005=1 : Homoscedasticity      49
Name: Result_test, dtype: int64